In [1]:
from copy import deepcopy
import nltk
import numpy as np
import pynlpl.lm.lm as lm
import MultiTree as MT

In [2]:
model = lm.ARPALanguageModel('lang_model/3-gram-3.ARPA', base_e = False)


In [34]:
model.scoreword("a", history = ["hello","i", "am"])

TypeError: can only concatenate list (not "tuple") to list

In [35]:
def pr_lang(x, y):
    return 10**(model.scoreword(x, history = tuple(y.word_hist)))

In [36]:
phon_probs_ = np.load("phoneme_probs.npy")
for i in range(phon_probs_.shape[0]):
    phon_probs_[i][39] /= 9
    row_sum = np.sum(phon_probs_[i])
    phon_probs_[i] /= row_sum

In [37]:
phon_probs = phon_probs_[30:]

In [38]:
phon_probs[:,39]

array([  9.59717989e-01,   9.68377888e-01,   3.29333633e-01,
         8.17908067e-03,   4.35292954e-03,   8.35012019e-01,
         1.46086991e-01,   3.62522365e-03,   8.42471361e-01,
         3.20656031e-01,   2.05458775e-01,   2.26448983e-01,
         2.42476002e-03,   9.35930789e-01,   9.87574339e-01,
         7.66921206e-04,   9.30095255e-01,   8.67197752e-01,
         4.94681925e-01,   1.04227533e-04,   3.59074116e-01,
         9.46949482e-01,   3.51551026e-01,   9.76616889e-03,
         6.86805986e-04,   7.51005709e-01,   3.90136689e-01,
         5.34172282e-02,   8.53094995e-01,   6.19522870e-01,
         3.91584914e-03,   9.80124652e-01,   3.04340152e-04,
         9.31864321e-01,   9.25073385e-01,   7.03214228e-01,
         8.30477178e-01,   4.56601828e-01,   3.12435366e-02,
         5.11703432e-01,   6.79209292e-01,   6.70927926e-04,
         8.31142008e-01,   3.21187399e-04,   9.41528082e-01,
         3.03302109e-01,   1.86888187e-03,   9.35616791e-01,
         1.65090978e-01,

In [39]:
from process_data_temp import interpret_probs
interpret_probs(phon_probs)

['-',
 u'W',
 '-',
 u'AH',
 '-',
 u'T',
 '-',
 u'IH',
 '-',
 u'Z',
 '-',
 u'DH',
 u'AH',
 '-',
 u'S',
 '-',
 u'TH',
 '-',
 u'OW',
 '-',
 u'D',
 '-',
 u'L',
 '-',
 u'AW',
 '-',
 u'M',
 '-',
 u'B',
 '-',
 u'AY',
 '-']

In [40]:
phon_dict = {MT.phonemes[k] : k for k in range(39)}
phon_dict["-"] = 39

In [41]:
def pr_ctc(phon, t):
    return phon_probs[t-1][phon_dict[phon]]

In [42]:
from decoding import *

In [43]:
T = len(phon_probs)
T

89

In [44]:
pr_nb_store = [{} for i in range(T)]
pr_b_store = [{} for i in range(T)]

def test_beam_search(T, p_ctc, p_lang, width, beta):
    
    def pr_nb(t, y):
        if y in pr_nb_store[t]:
            return pr_nb_store[t][y]
        else:
            return 0.
    
    def pr_b(t, y):
        if y in pr_b_store[t]:
            return pr_b_store[t][y]
        else:
            return 0.
        
    def pr_nb_add(t, y, val):
        if y in pr_nb_store[t]:
            pr_nb_store[t][y] += val
        else:
            pr_nb_store[t][y] = val
    
    def pr_b_add(t, y, val):
        if y in pr_b_store[t]:
            pr_b_store[t][y] += val
        else:
            pr_b_store[t][y] = val
        
    #extension probability of adding a phoneme k
'''
    WRONG!!
    def p_ext(k, y, t):
        p_last = pr_nb(t-1, y)
        if y.last_phoneme() != k:
            p_last += pr_b(t-1, y)
        return times([p_ctc(k,t), power(p_trans(k, y), beta), p_last])
'''
    def p_ext(k, y, t):
        if y.last_phoneme() == k:
            p_last = pr_b(t-1, y)
        else:
            p_last = pr_nb(t-1, y) + pr_b(t-1, y)            


        return times([p_ctc(k,t), 
                      power(p_trans(k,y),beta), 
                      p_last])
    
    #derived phoneme transition prob from language model
    def p_trans(k, y):
        if y.tree.lookup([k]) == None:
            return 0.
        
        denom = np.sum(np.asarray([p_lang(x, y) for x in y.tree.get_leaves()]))
        numer = np.sum(np.asarray([p_lang(x, y) for x in y.tree.lookup([k]).get_leaves()]))
        return divide(numer, denom)
    
    #penalty probability for converting to a homophone
    def p_homophone(word, y):
        denom = np.sum(np.asarray([p_lang(x, y) for x in y.tree.get_leaves()]))
        numer = p_lang(word, y)
        return divide(numer, denom)
    #initialising non-blank prob
    empty_str = DecodeInfo()
    pr_nb_store[0][empty_str] = 1.
    pr_b_store[0][empty_str] = 0.
    

    
    #most probable sequences
    B = [empty_str]
    
    for t in range(1, T+1):
            B_hat = get_best_paths(B, pr_b, pr_nb, width, t-1)
            print "\n".join(str(elem) + " : {}".format(pr_b(t-1, elem) + pr_nb(t-1, elem)) for elem in B_hat) + "\n\n"
            B = []
            for y in B_hat:
                if not y.empty():
                    pr_nb_store[t][y]= times([pr_nb(t-1, y), p_ctc(y.last_phoneme(), t)])
                    
                    #dont think we should have this - paths which decode to the same thing are accounted for in the
                    #loop below
#                     if y.one_fewer() in B_hat:
#                         pr_nb_store[t][y] += p_ext(y.last_phoneme(), y.one_fewer(), t)

                pr_b_store[t][y] = times(pr_b(t-1,y) + pr_nb(t-1, y), p_ctc("-", t))
                
                B.append(y)
                
                for k in MT.phonemes:

                    #if adding a phoneme k is valid
                    if k in y.tree.branches:

                        pr_plus = p_ext(k, y, t)

                        #initialising y_new = y+k
                        y_new = DecodeInfo(orig = y)
                        y_new.add_phon(k)

                        conversion_penalty_prob = 0.
                        #handles conversion from phoneme string to word
                        for word in y_new.tree.leaf:

                            #converted_y converts phoneme part of y_new into a new word
                            converted_y = DecodeInfo()
                            converted_y.word_hist = deepcopy(y_new.word_hist)
                            converted_y.word_hist.append(word)
                            pr_b_add(t, converted_y, 0.)
                            pr_nb_add(t, converted_y, times(pr_plus, power(p_homophone(word, y_new), beta)))
                            B.append(converted_y)
                            conversion_penalty_prob += p_homophone(word, y_new)

                        #penalises non-converted string
                        pr_nb_add(t, y_new, times(pr_plus, power(1.0 - conversion_penalty_prob, beta)))
                        B.append(y_new)

    return (B, pr_b_store, pr_nb_store)#get_best_paths(B, pr_b_store, pr_nb_store, 1)




In [45]:
beta = 1.25

In [ ]:
test_beam_search(T, pr_ctc, pr_lang, 15, beta)

| : 1.0


| : 0.959717990035
|S : 0.000160945277096
|F : 8.63466030884e-05
|HH : 3.67681010005e-05
|AH : 3.04627939209e-05
|V : 2.98086634053e-05
|T : 2.80442534697e-05
|B : 1.89050190294e-05
|D : 1.81221712939e-05
|Z : 1.64633886786e-05
|R : 1.6114505135e-05
|IH : 1.43662807159e-05
|G : 1.24643030703e-05
|AO : 1.15007551917e-05
|AE : 1.14053720395e-05


| : 0.929369681538
|S : 0.000155926807575
|F : 8.37506805101e-05
|HH : 3.5627125559e-05
|AH : 2.98524964357e-05
|V : 2.89492532792e-05
|T : 2.71739898669e-05
|P : 2.33191257506e-05
|B : 1.83254682297e-05
|D : 1.75544734415e-05
|Z : 1.59677673178e-05
|R : 1.56114896148e-05
|IH : 1.39171707936e-05
|G : 1.20726900358e-05
|AO : 1.11688029275e-05


| : 0.306072706471
|W : 0.00421716733592
|M : 0.000408952206802
|S : 5.13520309047e-05
|F : 2.75825052092e-05
|K : 1.29042627923e-05
|OW : 1.28374494533e-05
|HH : 1.17336819992e-05
|AH : 9.83527018209e-06
|V : 9.53857409376e-06
|T : 8.94931954313e-06
|P : 7.89766913636e-06
|N : 7.49544018398e-06


In [73]:
pr_b_store[2][DecodeInfo()]

1.8625862530573387

In [63]:
pr_nb_store = [{}]*(T+1)
pr_b_store = [{}]*(T+1)


In [74]:
DecodeInfo().empty()

True

In [21]:
power(-1,0)

nan